# Cartpole DQN

In [2]:
#pip install gym

  Stored in directory: C:\Users\vrn763\AppData\Local\pip\Cache\wheels\57\b0\13\4153e1acab826fbe612c95b1336a63a3fa6416902a8d74a1b7
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install keras

Note: you may need to restart the kernel to use updated packages.


#### import dependencies

In [6]:
#pip install tensorflow

  Stored in directory: C:\Users\vrn763\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: C:\Users\vrn763\AppData\Local\pip\Cache\wheels\5c\2e\7e\a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: C:\Users\vrn763\AppData\Local\pip\Cache\wheels\ee\98\38\46cbcc5a93cfea5492d19c38562691ddb23b940176c14f7b48
Successfully built termcolor gast absl-py
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import gym
import numpy as np
import tensorflow as tf
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os

Using TensorFlow backend.


In [3]:
env = gym.make('CartPole-v1')

In [6]:
env

<TimeLimit<CartPoleEnv<CartPole-v1>>>

In [6]:
state_size = env.observation_space.shape[0]
state_size

4

In [7]:
action_size = env.action_space.n
action_size

2

In [8]:
batch_size = 32

In [9]:
n_episodes = 1001

In [10]:
output_dir = 'model_output/cartpole'

In [11]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Define agent

In [28]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=2000) # "Deep Q Learning Networks" 33:30
        
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.learning_rate =  0.001
        
        self.model = self._build_model()
        
    def _build_model(self):
        
        model = Sequential()
        
        model.add(Dense(24, input_dim = self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
        
    def replay(self, batch_size):
        
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)
            
    def save(self, name):
        self.model.save_weights(name)

In [29]:
agent = DQNAgent(state_size, action_size)

#### Interact with environment

In [31]:
done = False
for e in range(n_episodes):
    
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000):
        
        env.render()

        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
        
        reward = reward if not done else -10
        
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
        
    if e % 50 == 0:
        agent.save(output_dir + "weights_" +  '{:04d}'.format(e) + ".hdf5")
            

episode: 0/1001, score: 104, e: 0.01
episode: 1/1001, score: 199, e: 0.01
episode: 2/1001, score: 199, e: 0.01
episode: 3/1001, score: 199, e: 0.01
episode: 4/1001, score: 199, e: 0.01
episode: 5/1001, score: 199, e: 0.01
episode: 6/1001, score: 199, e: 0.01
episode: 7/1001, score: 199, e: 0.01
episode: 8/1001, score: 112, e: 0.01
episode: 9/1001, score: 146, e: 0.01
episode: 10/1001, score: 199, e: 0.01
episode: 11/1001, score: 199, e: 0.01
episode: 12/1001, score: 199, e: 0.01
episode: 13/1001, score: 199, e: 0.01
episode: 14/1001, score: 199, e: 0.01
episode: 15/1001, score: 114, e: 0.01
episode: 16/1001, score: 168, e: 0.01
episode: 17/1001, score: 199, e: 0.01
episode: 18/1001, score: 199, e: 0.01
episode: 19/1001, score: 199, e: 0.01
episode: 20/1001, score: 199, e: 0.01
episode: 21/1001, score: 199, e: 0.01
episode: 22/1001, score: 182, e: 0.01
episode: 23/1001, score: 199, e: 0.01
episode: 24/1001, score: 149, e: 0.01
episode: 25/1001, score: 199, e: 0.01
episode: 26/1001, scor

KeyboardInterrupt: 